In [12]:
import pandas as pd
import pymysql as pymysql

debug = True

start_date = '2017-01-01'
end_date = '2018-01-01'
resample_interval = '5T'

host = "mariadb.mmto.arizona.edu"
database = "measurements"
table = 'bump'
usr = "webuser"
pw = "Clear7Skies!"

conn = pymysql.connect(host=host, port=3306, user=usr, passwd=pw, db=database)

In [13]:
#  This gets the individual e-series thermocouples parameter names
def get_names(connection):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT ds_name FROM aaa_parameters WHERE ds_name LIKE 'cell_e_series%'"""
            df = pd.read_sql(sql, conn)
    except Exception as e:
        print(f"Error: {e}")
    return df

In [14]:
def get_data(connection, param, database, start_date, end_date):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT from_unixtime(timestamp/1000) as ts, value as {param} FROM {param} 
                  WHERE timestamp >= UNIX_TIMESTAMP('{start_date}') * 1000
                  AND timestamp < UNIX_TIMESTAMP('{end_date}') * 1000;"""
            # Using a median filter to remove spikes.  Modify as you wish.
            df = pd.read_sql(sql, conn, index_col='ts').resample(resample_interval).median()
    except Exception as e:
        print(f"Error: {e}")
    return df

In [15]:
df_names = get_names(conn)
df_names

,ds_name
0,cell_e_series_backplate_C
1,cell_e_series_backplate_max_C
2,cell_e_series_backplate_min_C
3,cell_e_series_carrier_offset_C
4,cell_e_series_carrier_reference_C
5,cell_e_series_chamber_ambient_C
6,cell_e_series_count
7,cell_e_series_crate_status
8,cell_e_series_diff_C
9,cell_e_series_frontplate_C


In [16]:
drop_list = [
    "cell_e_series_update",
    "cell_e_series_unit_status",
    "cell_e_series_timestamp",
    "cell_e_series_count",
    "cell_e_series_crate_status",
    "cell_e_series_ijb_raw",
    "cell_e_series_interval",
    "cell_e_series_restarted",
    "cell_e_series_server_status"
]

In [17]:
df_all = None
for (idx, name) in df_names.itertuples():
    if name not in drop_list:
        if debug:
            print(f"Getting {name} values...")
        df = get_data(conn, name, database, start_date, end_date)
        if df_all is None:
            df_all = df
        else:
            if df is not None:
                df_all = df_all.join(df, how='inner')

Getting cell_e_series_backplate_C values...
Getting cell_e_series_backplate_max_C values...
Getting cell_e_series_backplate_min_C values...
Getting cell_e_series_carrier_offset_C values...
Getting cell_e_series_carrier_reference_C values...
Getting cell_e_series_chamber_ambient_C values...
Getting cell_e_series_diff_C values...
Getting cell_e_series_frontplate_C values...
Getting cell_e_series_frontplate_max_C values...
Getting cell_e_series_frontplate_minus_backplate_C values...
Getting cell_e_series_frontplate_minus_midplate_C values...
Getting cell_e_series_frontplate_min_C values...
Getting cell_e_series_in_front_of_primary_C values...
Getting cell_e_series_lower_plenum_C values...
Getting cell_e_series_lp_rms_C values...
Getting cell_e_series_manifold_air_temp_C values...
Getting cell_e_series_mean_C values...
Getting cell_e_series_midplate_C values...
Getting cell_e_series_midplate_max_C values...
Getting cell_e_series_midplate_minus_backplate_C values...
Getting cell_e_series_mi

In [19]:
df_all.tail()

,cell_e_series_backplate_C,cell_e_series_backplate_max_C,cell_e_series_backplate_min_C,cell_e_series_carrier_offset_C,cell_e_series_carrier_reference_C,cell_e_series_chamber_ambient_C,cell_e_series_diff_C,cell_e_series_frontplate_C,cell_e_series_frontplate_max_C,cell_e_series_frontplate_minus_backplate_C,...,cell_e_series_tc91_C,cell_e_series_tc92_C,cell_e_series_tc93_C,cell_e_series_tc94_C,cell_e_series_tc95_C,cell_e_series_tc96_C,cell_e_series_tc97_C,cell_e_series_tc98_C,cell_e_series_tc99_C,cell_e_series_tc9_C
ts,,,,,,,,,,,,,,,,,,,,,
2017-12-31 23:35:00,7.61740,7.76180,7.52910,0.0,0.0,7.68430,-0.08270,7.61870,7.75150,-0.00190,...,7.55500,7.56020,7.5395,7.45150,7.59130,7.56530,7.48250,7.65840,7.48270,7.6016
2017-12-31 23:40:00,7.61335,7.74645,7.51365,0.0,0.0,7.78525,-0.18165,7.61485,7.72835,0.00195,...,7.54725,7.57065,7.5395,7.47480,7.60935,7.57575,7.51365,7.66885,7.50335,7.5861
2017-12-31 23:45:00,7.61040,7.73610,7.52410,0.0,0.0,7.76730,-0.15880,7.61590,7.73110,0.00490,...,7.54480,7.57570,7.5655,7.49830,7.61720,7.58620,7.51370,7.66880,7.51900,7.5810
2017-12-31 23:50:00,7.61760,7.73620,7.52685,0.0,0.0,7.76985,-0.15880,7.62310,7.72070,0.00710,...,7.54490,7.58365,7.5707,7.51645,7.63800,7.59395,7.54485,7.67940,7.53190,7.5862
2017-12-31 23:55:00,7.61240,7.73620,7.52420,0.0,0.0,7.71550,-0.10060,7.62500,7.72070,0.01070,...,7.53960,7.59140,7.5449,7.50350,7.61730,7.59150,7.54490,7.66390,7.52940,7.5759


In [20]:
len(df_all)

104063

In [21]:
df_all.to_csv("../halcoll/data/2017_e_series.csv")